In [2]:
# Enable widget mode for interactive plotting in Jupyter Lab
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, spectrogram
from scipy.signal.windows import hamming
from datetime import datetime, timedelta
import time

In [3]:
# Load CSV file (substitute with your actual file path)
data = pd.read_csv('Experiment-5-Pump.csv')
threshold = -20

# Parse the datetime column with microsecond precision
time_utc = pd.to_datetime(data.iloc[:, 1], format='%Y-%m-%d %H:%M:%S.%f')
signal = data.iloc[:, 42].values  # Assuming the signal is in column 43

# Convert datetime to seconds relative to the first timestamp
time_seconds = (time_utc - time_utc.iloc[0]).dt.total_seconds()
new_time_seconds = np.arange(time_seconds.min(), time_seconds.max(), 2)  # Regular 2-second intervals
interpolated_signal = np.interp(new_time_seconds, time_seconds, signal)

# High-pass filter with cutoff frequency of 0.01 Hz
cutoff_frequency = 0.002
Fs = 1 / 2  # Sampling frequency (1 sample every 2 seconds)
b, a = butter(4, cutoff_frequency / (Fs / 2), 'high')
filtered_signal = filtfilt(b, a, interpolated_signal)

# Apply STFT to the filtered signal
f, t, Sxx = spectrogram(filtered_signal, fs=Fs, window=hamming(128), nperseg=128, noverlap=120)
p_db = 10 * np.log10(np.abs(Sxx))

# Define frequency range of interest
freq_range = (f >= 0.05) & (f <= 0.25)
avg_power = p_db[freq_range, :].mean(axis=0)

# Create UTC time for plotting
new_time_utc = [time_utc.iloc[0] + timedelta(seconds=int(s)) for s in new_time_seconds]
t_utc = [new_time_utc[round(ti * Fs)] for ti in t]

In [4]:
# Maximum points to display
max_points = 1000

# Set up interactive plot with ipywidgets
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 8))

# Initialize line plots
line1, = ax1.plot([], [], 'b')
ax1.set_title('Interpolated Signal Over Time')
ax1.set_xlabel('Time (UTC)')
ax1.set_ylabel('Signal')
ax1.grid(True)

line2, = ax2.plot([], [], 'r')
ax2.set_title('Average Power in Frequency Range (0.05 - 0.25 Hz)')
ax2.set_xlabel('Time (UTC)')
ax2.set_ylabel('Average Power (dB)')
ax2.grid(True)

# Initialize spectrogram plot
img = ax3.imshow(np.zeros_like(p_db), aspect='auto', origin='lower',
                 extent=[t_utc[0], t_utc[-1], f[0], f[-1]], cmap='viridis')
ax3.set_title('Spectrogram - Frequency Change Over Time')
ax3.set_xlabel('Time (UTC)')
ax3.set_ylabel('Frequency (Hz)')
fig.colorbar(img, ax=ax3)

# Real-time update function
def update_plot(i):
    start_idx = max(0, i - max_points)
    idx_range = slice(start_idx, i)
    
    # Update interpolated signal plot
    line1.set_data(new_time_utc[idx_range], interpolated_signal[idx_range])
    ax1.set_xlim(new_time_utc[start_idx], new_time_utc[i])
    ax1.set_ylim(np.min(interpolated_signal), np.max(interpolated_signal))
    
    # Update average power plot
    if i < len(avg_power):
        line2.set_data(new_time_utc[:i], avg_power[:i])
        ax2.set_xlim(new_time_utc[0], new_time_utc[i])
        ax2.set_ylim(np.min(avg_power), np.max(avg_power))
        
        # Mark detected failure times
        failure_indices = np.where(avg_power[:i] > threshold)[0]
        for failure_idx in failure_indices:
            ax2.axvline(new_time_utc[failure_idx], color='k', linestyle='--', linewidth=2)

    # Update spectrogram plot
    if i < len(t):
        img.set_data(p_db[:, :i])
        img.set_extent([t_utc[0], t_utc[i-1], f[0], f[-1]])
    
    fig.canvas.draw_idle()

# Run real-time update loop
for i in range(1, len(new_time_utc)):
    update_plot(i)
    time.sleep(0.1)  # Adjust pause time as needed

plt.show()

C:\Users\cjwattz\AppData\Local\Temp\1\ipykernel_7816\3708338491.py:52: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  img.set_extent([t_utc[0], t_utc[i-1], f[0], f[-1]])


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x00000176378C0720> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt

